In [7]:
%matplotlib inline
import numpy as np
import pandas as pd
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from random import sample


In [8]:
def readCsv(fileName):
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    df = pd.read_csv(fullFileName, encoding='utf-8', header = None,
                 comment='#', sep=',')
    
    return df

In [9]:
def createData():
    df = readCsv('1_Cov.csv')
    meanPositive = readCsv('1_m_1.csv').dropna(axis=1, how='all').as_matrix()
    meanNegative = readCsv('1_m_0.csv').dropna(axis=1, how='all').as_matrix()
    meanPositive = meanPositive[0,:]
    meanNegative = meanNegative[0,:]
    
    cov = df.dropna(axis=1, how='all').as_matrix()
    
    distributionPositive = np.random.multivariate_normal(meanPositive, cov, 2000)
    distributionNegative = np.random.multivariate_normal(meanNegative, cov, 2000)
    
    distributionPositive = np.c_[distributionPositive, np.ones(distributionPositive.shape[0])]
    distributionNegative = np.c_[distributionNegative, np.zeros(distributionNegative.shape[0])]
#     print('Positive', distributionPositive)
#     print('Negative', distributionNegative)
    distribution = np.concatenate((distributionPositive, distributionNegative), axis=0)
    np.random.shuffle(distribution)
    distributionTest = distribution[0:1200]
    distributionTrain = distribution[1200:] 
    print(distribution)
    print(distributionTest)
    print(distributionTrain)
    print('original', distribution.shape)
    print('train', distributionTrain.shape)
    print('test', distributionTest.shape)
    return (distribution, distributionTest, distributionTrain)


In [10]:
def toCsv(fileName, npArray):
    
    fullFileName = r'C:\Users\Owner\McGill\4thYear\COMP551\Assignments\Assignment2\Datasets\DS' + fileName
    np.savetxt(fullFileName, npArray, delimiter=',')


In [11]:
def initializeDS1():
    fullDistribution, distributionTest, distributionTrain = createData()
    toCsv('1_train.csv', distributionTrain)
    toCsv('1_test.csv', distributionTest)
    toCsv('1.csv', fullDistribution)
initializeDS1()


[[ 0.34202437  1.35147519  2.52792192 ...,  1.07567635 -2.23509516  0.        ]
 [ 2.27473906  0.72331772  2.55903595 ..., -0.32376621  0.98610758  0.        ]
 [ 6.95724059  5.88878613  4.8385471  ...,  2.88931056  3.85766229  0.        ]
 ..., 
 [ 2.64886532  2.46943597  1.72987447 ...,  4.30558941  3.64403896  0.        ]
 [ 2.51985751  1.39122851  1.11322351 ...,  0.13196815  2.28450757  1.        ]
 [-1.88003678 -0.35812245 -2.13434339 ..., -1.15754924  2.11947785  1.        ]]
[[ 0.34202437  1.35147519  2.52792192 ...,  1.07567635 -2.23509516  0.        ]
 [ 2.27473906  0.72331772  2.55903595 ..., -0.32376621  0.98610758  0.        ]
 [ 6.95724059  5.88878613  4.8385471  ...,  2.88931056  3.85766229  0.        ]
 ..., 
 [ 0.22473106 -0.26968278  2.15060049 ...,  1.85207128 -1.38497024  1.        ]
 [ 1.76623179  4.0809592   3.7035515  ...,  2.83982552  2.94995842  0.        ]
 [ 1.06299512 -0.13768842  2.46645962 ...,  0.38812707 -0.52804695  0.        ]]
[[-0.08914377  1.1347193

In [12]:
def visualizeStuff():
    mean = [10, 10, 10]
    cov = [[1, 0, 0], [0, 1, 0], [0, 0, 1]] 
    x, y, z = np.random.multivariate_normal(mean, cov, 10000).T

    fig = plt.figure(figsize=(20,8))
    ax = fig.add_subplot(111, projection='3d')

    plt.scatter(x, y, z, c='red')
    plt.axis('equal')
    plt.show()
# visualizeStuff()

In [25]:
def runLDA():
    dfTrain = readCsv('1_train.csv')
    N1, N2, pi, U1, U2 = np.zeros(5)
    Y = dfTrain[20].as_matrix()
    dfTrain = dfTrain.iloc[:,0:20]
    S = np.zeros((20,20))
    S1 = np.zeros((20,20))
    S2 = np.zeros((20,20))
    N1 = np.count_nonzero(Y)
    N2 = 2800 - N1
    pi = N1/(N1+N2)
#     print(S)
    
    for i in range(0, len(dfTrain)):
        U1 = U1 + Y[i]*dfTrain.iloc[i,:].as_matrix()
        U2 = U2 + (1-Y[i])*dfTrain.iloc[i,:].as_matrix()
    U1 = U1/N1
    U2 = U2/N2
    
    print(dfTrain.iloc[0,:].as_matrix())
    print(U1)
    print(np.subtract(dfTrain.iloc[0,:].as_matrix(), U1).size)
#     print(np.zeros(20).size)

    for i in range(0, len(dfTrain)):
        xSubMean = np.zeros(20)
        if (Y[i]==1):
            xSubMean = np.subtract(dfTrain.iloc[0,:].as_matrix(), U1)   
            S1 = S1 + np.outer(xSubMean, xSubMean)
        elif (Y[i]==0):
            xSubMean = np.subtract(dfTrain.iloc[0,:].as_matrix(), U2)
            S2 = S2 + np.outer(xSubMean, xSubMean)
        
        S = S + np.outer(xSubMean, xSubMean)
        
    print(S)
        
#     print(U1)
#     print(U2)
    
#     for i in range(0)

#     print(dfTrain)
    test = dfTrain.iloc[0,:].as_matrix()
    
#     print(test)
#     print(test.transpose())
#     print(np.dot(test,test))
#     print(np.outer(test, test).size)
#     for i in range(0, len(dfTrain)):
#         print(dfTrain.iloc[i,20])
#         U1 += dfTrain.iloc[i,20]*
runLDA()


[-0.08914377  1.13471938  1.29142313 -0.61206382  0.93574244  1.13285524
  1.71492989 -0.66893323  1.1344586   0.20889852  0.43569528 -0.54773379
 -1.4170407   1.22573711  1.95883942  1.13204014 -0.19855062 -0.25160205
  0.8753283   0.99120692]
[ 1.92486305  1.88574537  1.91903303  1.9023876   1.87220534  1.92675551
  1.90384851  1.93631247  1.92383374  1.88307188  1.93366121  1.8939727
  1.8565985   1.92555162  1.90964703  1.86961943  1.91901221  1.90444018
  1.92062611  1.86677443]
20
[[  8.21891289e+03   2.49283473e+03   1.64966789e+03   1.05958724e+04
    3.27934742e+03   2.50514017e+03  -2.52127622e+02   1.09517173e+04
    2.41708431e+03   6.77324398e+03   5.78023029e+03   1.03031212e+04
    1.43034188e+04   2.00704994e+03  -1.44417227e+03   2.29924396e+03
    8.61675164e+03   8.94599631e+03   3.72190040e+03   3.04116004e+03]
 [  2.49283473e+03   8.45085178e+02   6.42173493e+02   3.16170192e+03
    1.07900673e+03   8.74124986e+02   8.27057452e+01   3.27191461e+03
    8.58244378e+0